In [1]:
"""
Compute the left Frobenius morphism σ^i ⊗ 1 of `a`.
"""
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

"""
Compute the right Frobenius morphism 1 ⊗ σ^i of `a`.
"""
def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))
"""
Compute the right "norm" of `x`: the product of (σ^(a×i) ⊗ 1)(x) for i from 1 to b/a.
"""
def norm_r(x, a, b):
    d = b//a
    return product(frob_r(x, a*j) for j in range(d))

"""
Compute the `n`-th root of `x`.
"""
def nth_root_r(x, n):
    A = x.parent()
    k = A.base_ring().base_ring()
    C = k.extension(A.modulus().change_ring(k), 'x')
    L = [k(a) for a in x.list()]
    y = C(L) if (len(L) > 1) else C(L[0])
    return A(y.nth_root(n))

"""
Compute the degree of the smallest extension of `k` containing a `n`-th root of unity.
"""
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

In [2]:
"""
Express `elem` in the base `newbase`. Outputs a column matrix. 

    Remark: if `elem` is in a sub-vector space with base `newbase`, the function
    still returns the coordinates, provided that the degree `d2` of the subspace
    is given.
"""
def base_change(elem, newbase, d2 = None):
    k = elem.base_ring()
    d1 = elem.parent().degree()
    
    if d2 == None:
        d2 = d1
        
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (newbase^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S = MatrixSpace(k, d1, 1)
    B = S(elem.list())
    X = A.solve_right(B)
    return X

In [13]:
"""
Check if `a` is a solution of the Hilbert 90 Problem with `zeta`.
"""
def is_h90(a, zeta):
    return a != 0 and frob_l(a) == zeta * a

"""
Find a solution of the Hilbert 90 Problem with `z` in the algebra `A`.

    Remark: If `more` is set to True, it ensures that the solution x
    verifies x^l = constant, where the constant is well-chosen.
"""
def solve_h90(A, z = None, more = False, l = None):
    x = A(1)
    n = A.base_ring().degree()
    if l == None:
        l = n
    if z == None:
        z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        y = x
        zinv = z^-1
        zz = A.one()
        for i in range(1, n):
            zz *= zinv
            y = frob_l(y)
            x += y*zz
    
    if not more:
        return x
    else:
        k = A.base_ring().base_ring()
        t = (A.gen()^A.degree())/(x^l)
        return x*nth_root_r(t, l)

In [4]:
def basis_matrix(a, n = None):
    K = a.parent()
    m = K.degree()
    
    if n == None:
        n = m
    
    k = K.prime_subfield()
    S = MatrixSpace(k, m, n)()
    for j in range(n):
        L = (a^j).polynomial().list()
        i = 0
        for l in L:
            S[i, j] = l
            i += 1
    return S

def fflist(x):
    L = x.polynomial().list()
    l = len(L)
    k = x.parent()
    d = k.degree()
    if l < d:
        L += (d-l)*[k()]
        
    return L

def compute_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = b.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, a.parent().degree(), 1)

    return lambda x : K((C*S(fflist(x))).column(0))

def compute_emb(h1, h2):
    
    A, B = h1.parent(), h2.parent()
    a, b = A.degree(), B.degree()
    l, m = h1.base_ring().degree(), h2.base_ring().degree()

    p = h1.parent().characteristic()
    
    zeta_l, ZETA_L = A.gen()^((p^a-1)/l), B.gen()^((p^b-1)/l)

    u = base_change(h1, zeta_l)[0, 0]
    v = base_change(h2, ZETA_L, a)[0, 0]
    return compute_map(u, v)

In [5]:
def compute_inv_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = a.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, b.parent().degree(), 1)

    return lambda x : K(C.solve_right(S(fflist(x))).column(0))

def compute_inv_emb(h1, h2):
    
    A, B = h1.parent(), h2.parent()
    a, b = A.degree(), B.degree()
    l, m = h1.base_ring().degree(), h2.base_ring().degree()

    p = h1.parent().characteristic()
    
    zeta_l, ZETA_L = A.gen()^((p^a-1)/l), B.gen()^((p^b-1)/l)

    u = base_change(h1, zeta_l)[0, 0]
    v = base_change(h2, ZETA_L, a)[0, 0]
    return compute_inv_map(u, v)

In [6]:
def factor_refinement(l, m):
    
    t = gcd(l, m)
    u, v = m//t, t
    d = gcd(u, v)
    
    while d != 1: 
        u, v = u//d, v*d
        d = gcd(u, v)
    
    return v, u
"""
Return N, e, t such that Norm( (alpha_N)^e ) = (a_l)^t · alpha_l
and m | N.
"""
def height(p, l, m):
    k = GF(p)
    a, b = cyclo_deg(k, l), cyclo_deg(k, m)
    S = p^a-1
    x, y = m//l, (p^b-1)//S
    
    xl, (yl, ym) = factor_refinement(l, x)[0], factor_refinement(l, y)
    t = (yl/xl).denominator()
    
    u = factor_refinement(l, m)[1]
        
    N = l*yl*t*u
    f = ym/u
    
    e = inverse_mod(f.numerator(), l)
    
    return N, t*e*f.denominator(), ((f.numerator()*e)//l)%S

In [7]:
def make_context(n):
    
    global p, ZETAS, H90_ELEMENTS, EMBEDDINGS
    
    ZETAS = dict()
    H90_ELEMENTS = dict()
    EMBEDDINGS = dict()
    p = n
    
def compute_zetas(n):
    k = GF(p^n)
    zeta_pn = k.multiplicative_generator()
    
    for d in n.divisors():
        ZETAS[d] = (zeta_pn^((p^n-1)//(p^d-1))).minpoly()

In [8]:
def embed(K, L):
    
    k = K.prime_subfield()
    l, m = K.degree(), L.degree()
    a, b = cyclo_deg(k, l), cyclo_deg(k, m)
    
    assert b in ZETAS.keys(), "The corresponding zeta is not computed: not implemented"
    
    if (l, m) in EMBEDDINGS.keys():
        return EMBEDDINGS[(l, m)]
    
    if l in H90_ELEMENTS.keys():
        hl = H90_ELEMENTS[l]
    else:
        Al = PolynomialQuotientRing(GF(p^l)["x"], ZETAS[a].change_ring(GF(p^l)))
        hl = solve_h90(Al, Al.gen()^((p^a-1)//l), True)
        H90_ELEMENTS[l] = hl
    
    if m in H90_ELEMENTS.keys():
        hm = H90_ELEMENTS[m]
    else:
        Am = PolynomialQuotientRing(GF(p^m)["x"], ZETAS[b].change_ring(GF(p^m)))
        hm = solve_h90(Am, Am.gen()^((p^b-1)//m), True)
        H90_ELEMENTS[m] = hm

    if a == b:
        f = compute_emb(hl, hm^(m//l))
        EMBEDDINGS[(l, m)] = f
        return f
    else:
        R = k['z']
        N, e, c = height(p, l, m)
        C = k.extension(ZETAS[b], 'z')
        ZA = C.gen()^((p^b-1)//(p^a-1))
        
        if N > m:
            if N in H90_ELEMENTS.keys():
                hN = H90_ELEMENTS[N]
            else:
                AN = PolynomialQuotientRing(GF(p^N)["x"], ZETAS[b].change_ring(GF(p^N)))
                hN = solve_h90(AN, AN.gen()^((p^b-1)//N), True)
                H90_ELEMENTS[N] = hN

            al = (hN.parent())(R((hl^l).list()).subs(ZA))
            HL = (al^c)^-1 * norm_r(hN, a, b)^e
            g = compute_emb(hl, HL)
            inv = compute_inv_emb(hm, hN^(N//m))
            f = lambda x : inv(g(x))
            EMBEDDINGS[(l, m)] = f
            return f
        else:
            al = (hm.parent())(R((hl^l).list()).subs(ZA))
            HL = (al^c)^-1 * norm_r(hm, a, b)^e
            f = compute_emb(hl, HL)
            EMBEDDINGS[(l, m)] = f
            return f

# Tests

- **On triche** parce qu'on calcule d'abord les racines p^n-1 ièmes de l'unité par le haut, pour se simplifier la vie niveau compatibilité...
- On ne traite pas le cas Artin-Schreier

In [9]:
MAX = 36
p = 11
make_context(p)
compute_zetas(MAX)
factor(p^(MAX)-1)

2^4 * 3^3 * 5 * 7 * 13 * 19 * 37 * 61 * 1117 * 590077 * 1772893 * 3138426605161

In [10]:
k2.<x2> = GF(p^2)
k3.<x3> = GF(p^3)
k4.<x4> = GF(p^4)
k5.<x5> = GF(p^5)
k6.<x6> = GF(p^6)
k8.<x8> = GF(p^8)
k9.<x9> = GF(p^9)
k10.<x10> = GF(p^10)
k12.<x12> = GF(p^12)
k16.<x16> = GF(p^16)
k18.<x18> = GF(p^18)
k20.<x20> = GF(p^20)
k21.<x21> = GF(p^21)
k24.<x24> = GF(p^24)
k35.<x35> = GF(p^35)
k42.<x42> = GF(p^42)
k70.<x70> = GF(p^70)

In [11]:
f3_12 = embed(k3, k12)
f3_6 = embed(k3, k6)
f6_24 = embed(k6, k24)
f12_24 = embed(k12, k24)
f3_18 = embed(k3, k18)
f4_16 = embed(k4, k16)
f6_24 = embed(k6, k24)
f4_8 = embed(k4, k8)
f2_12 = embed(k2, k12)
f6_12 = embed(k6, k12)
f5_10 = embed(k5, k10)
f5_20 = embed(k5, k20)
f3_24 = embed(k3, k24)
f2_24 = embed(k2, k24)
f6_18 = embed(k6, k18)
f2_6 = embed(k2, k6)
f9_18 = embed(k9, k18)
f2_4 = embed(k2, k4)
f10_20 = embed(k10, k20)
f4_24 = embed(k4, k24)
f8_16 = embed(k8, k16)
f3_21 = embed(k3, k21)
f3_18 = embed(k3, k18)
f3_9 = embed(k3, k9)
f2_8 = embed(k2, k8)
f21_42 = embed(k21, k42)
f3_42 = embed(k3, k42)
f6_42 = embed(k6, k42)
f8_24 = embed(k8, k24)

In [12]:
assert f3_12(x3) == f6_12(f3_6(x3))
assert f6_24(f3_6(x3)) == f12_24(f3_12(x3))
assert f10_20(f5_10(x5)) == f5_20(x5)
assert f21_42(f3_21(x3)) == f3_42(x3)
assert f6_42(f3_6(x3)) == f3_42(x3)
assert f4_8(f2_4(x2)) == f2_8(x2)
assert f8_16(f4_8(x4)) == f4_16(x4)
assert f4_16(f2_4(x2)) == f8_16(f2_8(x2))
assert f6_18(f3_6(x3)) == f3_18(x3)
assert f12_24(f6_12(x6)) == f6_24(x6)
assert f9_18(f3_9(x3)) == f3_18(x3)
assert f6_24(f3_6(x3)) == f3_24(x3)
assert f6_12(f2_6(x2)) == f2_12(x2)
assert f12_24(f3_12(x3)) == f3_24(x3)
assert f6_24(f2_6(x2)) == f2_24(x2)
assert f8_16(f4_8(x4)) == f4_16(x4)
assert f8_24(f4_8(x4)) == f4_24(x4)
print("PASS")

PASS
